In [65]:

from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT


In [66]:
#借助包gym_super_mario_bros创建
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [14]:
print("env.action_space: ", env.action_space)

env.action_space:  Discrete(7)


下面这一步在最新版本的gym中可能会出现报错，问题在于env.step的返回值个数在不同的版本中不同。介于还在学习过程，建议先将gym的版本拉低

In [67]:

done = True
import cv2  
for step in range(7000):
    # if step % 100 == 0:
    #     print(step//100)
    # print(step)
    if done:
        state = env.reset()
    action = env.action_space.sample()
    state, reward, done, info = env.step(step //1000)
    cv2.imshow("game",state)

    if cv2.waitKey(1) &  0xff == ord("q"): # 在英文状态下，按下按s键 q 会关闭显示窗口    
        break
cv2.destroyAllWindows()
env.close()
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

强化学习执行step动作的返回一般是标准的，所以这里的返回同前面的倒立摆，动作执行后，环境返回四个变量（obj:新的观测值（一帧rgb图片）、reward：执行该动作获得的奖励值（在游戏中的得分）、done:回合是否结束（你控制的小人是不是死了，对应回合结束）、info:额外信息（比如'life': 2，剩余2条命等））

In [2]:
import gym_super_mario_bros
from gym.spaces import Box
from gym import Wrapper
from nes_py.wrappers import JoypadSpace#BinarySpaceToDiscreteSpaceEnv
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
import cv2
import numpy as np
import subprocess as sp

In [9]:
def process_frame(frame):
    if frame is not None:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY) #图像转换
        
        frame = cv2.resize(frame, (84, 84))[None, :, :] / 255. #裁剪合适大小，并归一化

        return frame
    else:
        return np.zeros((1,84, 84))

In [60]:

class CustomSkipFrame(Wrapper):
    def __init__(self, env, skip=4):
        super(CustomSkipFrame, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(4, 84, 84))
        self.skip = skip

    def step(self, action):
        total_reward = 0
        states = []
        state, reward, done, info = self.env.step(action)
        for i in range(self.skip):
            if not done:
                state, reward, done, info = self.env.step(action)
                total_reward += reward
                states.append(state)
            else:
                states.append(state)
        states = np.concatenate(states, 0)[None, :, :, :]
        return states.astype(np.float32), reward, done, info

    def reset(self):
        state = self.env.reset()
        states = np.concatenate([state for _ in range(self.skip)], 0)[None, :, :, :]
        return states.astype(np.float32)



In [55]:
class CustomReward(Wrapper):
    def __init__(self, env=None):
        super(CustomReward, self).__init__(env)
        self.observation_space = Box(low=0, high=255, shape=(1, 84, 84))
        self.curr_score = 0
        self.pre_x = 38 
    def step(self, action):
        state, reward, done, info = self.env.step(action) 
        state = process_frame(state)
        reward += (info["score"] - self.curr_score) / 50.
        
        reward += (info["x_pos"] - self.pre_x ) * 10.
        self.pre_x = info["x_pos"]
        # self.pre_x = info["x_pos"]
        self.curr_score = info["score"]
        if done:
            if info["flag_get"]:
                reward += 5000
            else:
                reward -= 50
        return state, reward/10. , done, info

    def reset(self):
        self.curr_score = 0
        return process_frame(self.env.reset())

In [56]:
#至此，我们完成了超级玛丽环境的自定义，封装如下：
def create_train_env(world, stage, action_type, output_path=None):
    env = gym_super_mario_bros.make("SuperMarioBros-{}-{}-v0".format(world, stage))
    if action_type == "right":
        actions = RIGHT_ONLY
    elif action_type == "simple":
        actions = SIMPLE_MOVEMENT
    else:
        actions = COMPLEX_MOVEMENT
    env = JoypadSpace(env, actions)
    env = CustomReward(env)
    env = CustomSkipFrame(env)
    return env, env.observation_space.shape[0], len(actions)

In [62]:
#测试一下
custom_env,_,_ = create_train_env(1,1,'simple')
print(custom_env)


<CustomSkipFrame<CustomReward<JoypadSpace<TimeLimit<SuperMarioBrosEnv<SuperMarioBros-1-1-v0>>>>>>


In [58]:
custom_env.reset()
done = True
flag =False 
step_ = 0 
n = 1
max_x = 38 

import cv2  
for step in range(50000):
    if done == True :
        custom_env.reset()
        
        if flag:
            print(f"成功了,距离上一次成功经过了{step-step_}步")
            step_ = step 
        x = info["x_pos"]
        if   x > max_x :
            print(f"失败了,第{n}次尝试,比最好成绩好了{x  - max_x},奖励值为{total_reward},目前最远距离为{x}")
            max_x = x
        total_reward = 0 
        n+=1 
    action = custom_env.action_space.sample() 
    states, reward, done, info = custom_env.step(action)
    total_reward += reward
    ############# 显示过程 ##################
    # flag = info["flag_get"]
    # print(info)
    # states = states[0]
    # for state in states:
    #     state.reshape(state.shape+(1,))
    #     cv2.imshow("test",state)
    # if cv2.waitKey(1) &  0xff == ord("q"): # 在英文状态下，按下按s键 q 会关闭显示窗口    
    #     break
    ##################################################
    
# cv2.destroyAllWindows()
custom_env.close()
cv2.destroyAllWindows()

失败了,第1次尝试,比最好成绩好了683,奖励值为1086.280000000002,目前最远距离为721
失败了,第5次尝试,比最好成绩好了1,奖励值为12.099999999999987,目前最远距离为722
失败了,第8次尝试,比最好成绩好了406,奖励值为21.19999999999999,目前最远距离为1128
失败了,第9次尝试,比最好成绩好了4,奖励值为20.999999999999964,目前最远距离为1132
失败了,第14次尝试,比最好成绩好了75,奖励值为24.09999999999996,目前最远距离为1207
失败了,第24次尝试,比最好成绩好了6,奖励值为25.699999999999967,目前最远距离为1213
失败了,第58次尝试,比最好成绩好了13,奖励值为26.19999999999998,目前最远距离为1226
失败了,第63次尝试,比最好成绩好了205,奖励值为33.00000000000003,目前最远距离为1431
失败了,第77次尝试,比最好成绩好了228,奖励值为40.399999999999984,目前最远距离为1659
失败了,第307次尝试,比最好成绩好了297,奖励值为48.599999999999994,目前最远距离为1956
失败了,第334次尝试,比最好成绩好了62,奖励值为48.5,目前最远距离为2018


In [63]:
custom_env.reset()
done = True
flag =False 
step_ = 0 
n = 1
max_x = 38 

import cv2  
for step in range(50000):
    if done == True :
        custom_env.reset()
        
        if flag:
            print(f"成功了,距离上一次成功经过了{step-step_}步")
            step_ = step 
        x = info["x_pos"]
        if   x > max_x :
            print(f"失败了,第{n}次尝试,比最好成绩好了{x  - max_x},奖励值为{total_reward},目前最远距离为{x}")
            max_x = x
        total_reward = 0 
        n+=1 
    action = custom_env.action_space.sample() 
    states, reward, done, info = custom_env.step(action)
    total_reward += reward
    ############# 显示过程 ##################
    # flag = info["flag_get"]
    # print(info)
    # states = states[0]
    # for state in states:
    #     state.reshape(state.shape+(1,))
    #     cv2.imshow("test",state)
    # if cv2.waitKey(1) &  0xff == ord("q"): # 在英文状态下，按下按s键 q 会关闭显示窗口    
    #     break
    ##################################################
    
# cv2.destroyAllWindows()
custom_env.close()
cv2.destroyAllWindows()

失败了,第1次尝试,比最好成绩好了198,奖励值为6.300000000000001,目前最远距离为236
失败了,第2次尝试,比最好成绩好了447,奖励值为20.59999999999996,目前最远距离为683
失败了,第4次尝试,比最好成绩好了215,奖励值为28.699999999999953,目前最远距离为898
失败了,第8次尝试,比最好成绩好了510,奖励值为49.499999999999964,目前最远距离为1408
失败了,第25次尝试,比最好成绩好了240,奖励值为60.800000000000104,目前最远距离为1648
失败了,第36次尝试,比最好成绩好了14,奖励值为56.60000000000004,目前最远距离为1662
失败了,第111次尝试,比最好成绩好了349,奖励值为71.10000000000008,目前最远距离为2011


发现一个问题在于如果不进行跳过帧这一操作时，马里奥有可能会出现一直小跳而跳不出较高的柱子的情况

教程中使用的是4帧的跳过，依然存在连续小跳的情况，于是将skip调整至8进行尝试。连续小跳的情况得到了较好的解决，但是存在马里奥往坑里跳的情况。

由于不熟悉reward的反馈机制，所以很难设置reward值。


知道原因了，上文所有的操作实现的只是随机动作，并不存在学习功能。

具有学习功能的在下章